<a href="https://colab.research.google.com/github/it-ces/PUJ-GeneticAnnealingNN/blob/main/project/Results-feature-selection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
!git clone https://github.com/it-ces/PUJ-GeneticAnnealingNN.git

Cloning into 'PUJ-GeneticAnnealingNN'...
remote: Enumerating objects: 58, done.
remote: Counting objects: 100% (58/58), done.
remote: Compressing objects: 100% (49/49), done.
remote: Total 58 (delta 16), reused 42 (delta 8), pack-reused 0 (from 0)
Receiving objects: 100% (58/58), 3.68 MiB | 11.70 MiB/s, done.
Resolving deltas: 100% (16/16), done.


In [29]:
!pip install deap

In [30]:
cd PUJ-GeneticAnnealingNN/project

/content/PUJ-GeneticAnnealingNN/project/PUJ-GeneticAnnealingNN/project/PUJ-GeneticAnnealingNN/project/PUJ-GeneticAnnealingNN/project/PUJ-GeneticAnnealingNN/project/PUJ-GeneticAnnealingNN/project


In [31]:
# Results feature selection
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from preprocessing import complete_vars
from preprocessing import ratios
from preprocessing import breakdown_vars
from preprocessing import dummies_ohe
from preprocessing import Xy
from preprocessing import std_z
from imblearn.under_sampling import RandomUnderSampler
from Genfet import Geafet
from gentools import PerformanceTest
from gentools import get_names
from ineq import Theil
from gentools import PerformanceTest
from gentools import get_names

df_train = pd.read_csv("Datapooled.csv")

VARS = ['Ganancia bruta', 'Ganancia (pérdida)','Ingresos de actividades ordinarias' , 'Costo de ventas', 'Patrimonio total',
     'Total pasivos', 'Total de activos', 'Ganancias acumuladas',  'Pasivos corrientes totales',  'Activos corrientes totales']

df_train.rename(columns={'Clasificación Industrial Internacional Uniforme Versión 4 A.C':'Sector'}, inplace=True)

df_train = df_train[VARS+[ 'event', 'Sector']]
print(df_train[df_train['event']==1].info())
df_train['complete-vars'] = complete_vars(df_train) #1 is that have all variables!
df_train =  df_train[df_train['complete-vars']==1] #filtering firms that have not financial information
print(df_train[df_train['event']==1].info())
df_train = ratios(df_train)
predictors =[ 'GPM', 'NPM', 'ROE','ROA', 'IR', 'DER', 'RSL', 'CR', 'Ax1', 'Ax2', 'Sector']
print(df_train[df_train['event']==1].info())
df_train.replace([np.inf,-np.inf], np.nan, inplace=True)
df_train.dropna(inplace=True)
df_train.drop(columns=['complete-vars'], inplace=True)
df_train = df_train[predictors + ['event']].copy()
df_exp = df_train.copy()
X, y = Xy(df_train, 'event')
cat,  nonormal, normal  = breakdown_vars(X)
nums = nonormal + normal
X = dummies_ohe(X, cat)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle = True, random_state = 666, stratify=y)
rus = RandomUnderSampler(random_state=123)
X_train, y_train = rus.fit_resample(X_train, y_train)
from preprocessing import standardize_X_test
X_test = standardize_X_test(X_train, X_test) # Apply the mean and std of X_test with info from X_train
X_train = std_z(nonormal + normal, X_train)

<class 'pandas.core.frame.DataFrame'>
Index: 771 entries, 4 to 26499
Data columns (total 12 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Ganancia bruta                      353 non-null    float64
 1   Ganancia (pérdida)                  353 non-null    float64
 2   Ingresos de actividades ordinarias  353 non-null    float64
 3   Costo de ventas                     330 non-null    float64
 4   Patrimonio total                    353 non-null    float64
 5   Total pasivos                       353 non-null    float64
 6   Total de activos                    353 non-null    float64
 7   Ganancias acumuladas                351 non-null    float64
 8   Pasivos corrientes totales          352 non-null    float64
 9   Activos corrientes totales          353 non-null    float64
 10  event                               771 non-null    float64
 11  Sector                              771 non-null

/usr/local/lib/python3.11/dist-packages/scipy/stats/_axis_nan_policy.py:586: UserWarning: scipy.stats.shapiro: For N > 5000, computed p-value may not be accurate. Current N is 16800.
  res = hypotest_fun_out(*samples, **kwds)


ImportError: cannot import name 'standardize_X_test' from 'preprocessing' (/content/PUJ-GeneticAnnealingNN/project/preprocessing.py)

In [ ]:
classifier = LogisticRegression(penalty=None, max_iter=1500, random_state=666)

Pop_sizes  = [45] # 45
generations = [50]
Mutations = [0.05, 0.09]
Tournaments = [3]
Cross = [0.85, 0.95]
probs_mate = [0.5]
probs_mutate = [0.05 , 0.09]
regist= Geafet(
       pop_sizes = Pop_sizes,
       generations = generations,
       mutations = Mutations,
       tournaments_sizes = Tournaments,
       crossovers=Cross,
       mate_indpb = probs_mate,
       mutate_indpb = probs_mutate,
       classifier = classifier,
       ineq_measure = Theil,
       ineq_min  = 0,
       limit_unchanged= 35,
       X_train=X_train,
       y_train=y_train,
       k_folds = 5,
       verbose=True)

In [ ]:
[hyper for hyper in regist]

In [ ]:
posible = [regist[hyper][1] for hyper in regist][0]

In [ ]:
[regist[hyper][0] for hyper in regist]

In [ ]:
np.array([regist[hyper][0] for hyper in regist]).argmax()

In [ ]:
fittest_  = [regist[hyper][1] for hyper in regist][np.array([regist[hyper][0] for hyper in regist]).argmax()]

In [ ]:
for sol in regist:
    print(PerformanceTest(solution = regist[sol][1],
                          classifier = classifier,
                          Xtrain=  X_train,
                          ytrain = y_train,
                          Xtest=  X_test,
                          ytest = y_test))

In [ ]:
PerformanceTest(solution = posible,
                          classifier = classifier,
                          Xtrain=  X_train,
                          ytrain = y_train,
                          Xtest=  X_test,
                          ytest = y_test)

In [ ]:
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.metrics import classification_report

# Forward Selection
sfs = SequentialFeatureSelector(estimator=classifier, n_features_to_select='auto', cv=5)
sfs.fit(X_train, y_train)
forwardvar = sfs.get_support()
sfs.get_feature_names_out()


# Backward Selection
sfs = SequentialFeatureSelector(estimator=classifier, n_features_to_select='auto', cv=5, direction='backward')
sfs.fit(X_train, y_train)
backwardvar = sfs.get_support()
sfs.get_feature_names_out()
X_train.columns[sfs.get_support()]


all = [1 for x in X_train.columns]
candidates = [all, fittest_, backwardvar, forwardvar]

# This function assest the performance of the model in testing

for candidate in candidates:
    print(PerformanceTest(solution = candidate,
                         classifier = classifier,
                         Xtrain=  X_train,
                         ytrain=  y_train,
                         Xtest=  X_test,
                         ytest = y_test))

In [ ]:
# Fittest
vars = get_names(fittest_, X_train)
classifier.fit(X_train.loc[:,vars], y_train)
preds = classifier.predict( X_test.loc[:,vars ])
print(classification_report(y_test, preds))
selectGA  = pd.DataFrame(classification_report(y_test, preds , output_dict=True)).iloc[:,0:2]

In [ ]:

# Forward Selection
sfs = SequentialFeatureSelector(estimator=classifier, n_features_to_select='auto', cv=2)
sfs.fit(X_train, y_train)
forwardvar = sfs.get_support()
sfs.get_feature_names_out()
classifier.fit(X_train.loc[:,forwardvar], y_train)
preds_forward = classifier.predict(X_test.loc[:,forwardvar])
print(classification_report(y_test, preds_forward))
forward  = pd.DataFrame(classification_report(y_test, preds_forward , output_dict=True)).iloc[:,0:2]

In [ ]:
# Backward Selection
sfs = SequentialFeatureSelector(estimator=classifier, n_features_to_select='auto', cv=2, direction='backward')
sfs.fit(X_train, y_train)
backwardvar = sfs.get_support()
classifier.fit(X_train.loc[:,backwardvar], y_train)
preds_backward = classifier.predict(X_test.loc[:,backwardvar])
print(classification_report(y_test, preds_backward))
backward  = pd.DataFrame(classification_report(y_test, preds_backward , output_dict=True)).iloc[:,0:2]


In [ ]:
models_tab = pd.concat([selectGA, forward, backward], axis=1)
cols_names =  pd.MultiIndex.from_tuples([('Genetic selection','No-Default'),("Genetic selection",'Deafult'),
              ("Forward selection",'No-Default'),('Forward selection','Default'),
              ("Backward selection",'No-Default'),('Backward selection','Default')])
models_tab.columns  = cols_names
models_tab = models_tab.style.set_table_styles([
   {'selector': 'th','props': [('text-align', 'center')]}]).format(precision=2)
models_tab.to_latex("feature-selection-logistic.tex")
models_tab

In [ ]:
vars

In [ ]:
X_train.loc[:,forwardvar].columns

In [ ]:
X_train.loc[:,backwardvar].columns